In [1]:
import time

from IPython.display import display
import arrow
import base_node
import base_node_rpc as bnr
import base_node_rpc.bootloader_driver
import dropbot as db
import hv_switching_board as hv
import hv_switching_board.driver
import logging; logging.basicConfig(level=logging.DEBUG)
import numpy as np
import platformio_helpers as pioh
import serial_device

# Show available COM ports.
display(serial_device.comports(only_available=True))

try:
    proxy.terminate()
except:
    pass
proxy = db.SerialProxy()
display(proxy.properties)
i2c_devices = proxy.i2c_scan()
print 'Found I2C devices at the following addresses: %s' % i2c_devices

# Find HV switching board firmware.
firmware_path = pioh.conda_bin_path().joinpath('hv-switching-board', 'v3_1',
                                               'firmware.hex')
print 'Found firmware: %s (%s)' % (firmware_path, arrow.get(firmware_path.mtime).to('local'))


def set_uuid(bootloader, value):
    '''
    Parameters
    ----------
    value : uuid.UUID
    '''
    config_bytes = bootloader.read_eeprom(0, base_node.driver.CONFIG_DTYPE.itemsize)
    config = np.fromstring(config_bytes.tobytes(), dtype=base_node.driver.CONFIG_DTYPE)[0]
    config['uuid'] = value.bytes
    bootloader.write_eeprom(0, config.tobytes())
    
    
def set_board_uuid(hv_board, value):
    bootloader = base_node_rpc.bootloader_driver.TwiBootloader(hv_board.proxy)
    hv_board.reboot_recovery()
    set_uuid(bootloader, value)
    bootloader.start_application()

    
def select_switching_board(proxy):
    i2c_devices = proxy.i2c_scan()
    switching_board_addrs = [address for address in i2c_devices if address not in [31, 80, 81]]

    # Create object to interact with I2C bootloader.
    bootloader = bnr.bootloader_driver.TwiBootloader(proxy, bootloader_address=0x29)
    if not switching_board_addrs:
        raise IOError('No switching boards found on bus.')
    elif bootloader.bootloader_address in switching_board_addrs:
        raise IOError('I2C bootloader found on bus at address 41.')
    else:
        selection = None
        while selection not in switching_board_addrs:
            print ('Please select one of the following switching boards into '
                   'recovery mode (or enter "q" to quit): %s' %
                   switching_board_addrs)

            selection = raw_input()
            if selection == 'q':
                return
            try:
                address = int(selection)
                return hv_switching_board.HVSwitchingBoard(proxy, address=address)
            except:
                pass

            
def write_firmware(bootloader, switching_board, firmware_path):
    if switching_board is not None:
        # Reboot switching board into recovery bootloader.
        switching_board.reboot_recovery()
        
    time.sleep(.2)
    bootloader.write_firmware(firmware_path)
    logging.info('Launch firmware')
    bootloader.start_application()
    # Wait for switching board firmware to boot.
    time.sleep(.2)
    i2c_devices = bootloader.proxy.i2c_scan()
    print 'Found I2C devices at the following addresses: %s' % i2c_devices

    
while True:
    try:
        switching_board = select_switching_board(proxy)
        print 'Selected: %s' % switching_board.uuid
        break
    except IOError:
        if 41 in proxy.i2c_scan():
            print 'Switching board found in recovery mode at I2C address 41'
            bootloader = bnr.bootloader_driver.TwiBootloader(proxy, bootloader_address=41)
            bootloader.start_application()
            time.sleep(.1)
        else:
            raise
        
bootloader = bnr.bootloader_driver.TwiBootloader(proxy, bootloader_address=41)

,descriptor,hardware_id,vid,pid,available
port,,,,,
COM15,USB Serial Device (COM15),USB VID:PID=16C0:0483 SER=3134890 LOCATION=1-1,16c0,0483,True


DEBUG:trollius:Using selector: SelectSelector
DEBUG:base_node_rpc.async:`ProactorEventLoop` required, not `<class 'trollius.windows_events._WindowsSelectorEventLoop'>`loop in background thread.
DEBUG:base_node_rpc.async:Execute new loop in background thread.
DEBUG:trollius:Using proactor: IocpProactor
DEBUG:base_node_rpc.async:`ProactorEventLoop` required, not `<class 'trollius.windows_events._WindowsSelectorEventLoop'>`loop in background thread.
DEBUG:base_node_rpc.async:Execute new loop in background thread.
DEBUG:trollius:Using proactor: IocpProactor
DEBUG:base_node_rpc.proxy:Attempt to connect to device on port COM15 (baudrate=115200)
DEBUG:serial_device.threaded:Open `COM15` and monitor connection status
DEBUG:serial_device.threaded:connection_made: `COM15` `<ReaderThread(Thread-13, started daemon 8552)>`
DEBUG:base_node_rpc.proxy:Wait for connection to port COM15
INFO:base_node_rpc.proxy:Successfully connected to dropbot on port COM15


base_node_software_version                                    0.48.1rc1
package_name                                                    dropbot
display_name                                                    DropBot
manufacturer                                              Sci-Bots Inc.
url                           http://gitlab.com/sci-bots/dropbot.py.git
software_version                                 1.48+49.g5f3aa88.dirty
dtype: object

Found I2C devices at the following addresses: [31 32 33 34 80 81]
Found firmware: C:\Users\chris\Miniconda2-x86\envs\db-dev-copy\share\platformio\bin\hv-switching-board\v3_1\firmware.hex (2017-11-24T12:51:00-05:00)
Please select one of the following switching boards into recovery mode (or enter "q" to quit): [32, 33, 34]
32
Selected: 651d3f89-ccd0-4ed6-b473-1adb680861bb


-----------------------------------------------


# Flash firmware

In [ ]:
write_firmware(bootloader, switching_board, firmware_path)

-----------------------------------------------


# Set switching board UUID

In [ ]:
import uuid

assert(switching_board is not None)
hv_uuid = uuid.UUID('XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX')
set_board_uuid(switching_board, hv_uuid)
time.sleep(.1)
print 'Set UUID to:', switching_board.uuid

-----------------------------------------------


# Set switching board I2C address

In [ ]:
assert(switching_board is not None)
switching_board.set_i2c_address(32)